# Notebook to Write Qualtrics Surveys

In [1]:
import pandas as pd
import numpy as np
import simple_search_func as ss
import re
import csv
import json

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
# Load up the qsf!
#with open('Query_Template.qsf', 'r') as file:
with open('Query_Template.qsf', 'r') as file:
    data = json.load(file)

In [ ]:
choices_dict = {
    'love': ['hello', 'hi', 'are you', 'darling', 'sweetheart', 'honey', 'romance', 'affection', 'passion', 'devotion', 
             'adore', 'cherish', 'fondness', 'embrace', 'beloved', 'cuddle', 'valentine', 'kiss', 'heart', 'soulmate'],
    'sadness': ['meow', 'meow meow', 'meowwww', 'tears', 'gloom', 'despair', 'melancholy', 'sorrow', 'blue', 'grief', 
                'heartache', 'mourn', 'loneliness', 'misery', 'pain', 'regret', 'weep', 'hurt', 'lost', 'downcast'],
    'kiss': ['smoocher', 'kombucher', 'peck', 'smooch', 'liplock', 'mwah', 'pucker', 'osculation', 'romantic', 'passionate',
             'sweet', 'gentle', 'soft', 'warm', 'butterfly', 'cheek', 'forehead', 'eskimo', 'lovebite', 'chaste'],
    'happiness': ['joy', 'smile', 'laughter', 'cheer', 'delight', 'bliss', 'glee', 'ecstatic', 'merry', 'euphoria', 
                  'sunny', 'content', 'radiant', 'jubilant', 'bubbly', 'grateful', 'optimistic', 'carefree', 'vivacious', 'buoyant'],
    'anger': ['rage', 'fury', 'wrath', 'outrage', 'resentment', 'bitterness', 'spite', 'grudge', 'irritation', 'annoyance',
              'exasperation', 'hostility', 'hatred', 'indignation', 'fuming', 'seething', 'mad', 'infuriated', 'irate', 'tempest'],
    'fear': ['terror', 'panic', 'dread', 'fright', 'horror', 'alarm', 'anxiety', 'nervous', 'uneasy', 'apprehension',
             'timid', 'startle', 'tremble', 'phobia', 'shiver', 'quiver', 'spooked', 'petrified', 'chill', 'paranoia'],
    'surprise': ['shock', 'amazement', 'astonishment', 'wonder', 'startled', 'stunned', 'flabbergasted', 'aghast', 'dumbfounded', 'jolted',
                 'marvel', 'unexpected', 'perplexed', 'bewildered', 'gobsmacked', 'astounded', 'overwhelmed', 'speechless', 'baffled', 'staggered'],
    'friendship': ['bond', 'companion', 'ally', 'trust', 'loyal', 'camaraderie', 'mate', 'bro', 'sisterhood', 'brotherhood',
                   'affection', 'partnership', 'support', 'understanding', 'team', 'respect', 'connect', 'harmony', 'together', 'kindred'],
    'jealousy': ['envy', 'resent', 'covet', 'green-eyed', 'possessive', 'suspicion', 'mistrust', 'distrust', 'bitterness', 'spite',
                 'comparison', 'insecurity', 'longing', 'competitiveness', 'inferiority', 'grudge', 'fuming', 'anger', 'watchful', 'wary'],
    'nostalgia': ['memories', 'past', 'bittersweet', 'longing', 'reminisce', 'yearning', 'flashback', 'old days', 'sentimental', 'melancholy',
                  'childhood', 'throwback', 'wistful', 'retro', 'antique', 'classic', 'golden days', 'vintage', 'heartwarming', 'timeless'],
    'fear2': ['terror', 'panic', 'dread', 'fright', 'horror', 'alarm', 'anxiety', 'nervous', 'uneasy', 'apprehension',
             'timid', 'startle', 'tremble', 'phobia', 'shiver', 'quiver', 'spooked', 'petrified', 'chill', 'paranoia'],
    'surprise2': ['shock', 'amazement', 'astonishment', 'wonder', 'startled', 'stunned', 'flabbergasted', 'aghast', 'dumbfounded', 'jolted',
                 'marvel', 'unexpected', 'perplexed', 'bewildered', 'gobsmacked', 'astounded', 'overwhelmed', 'speechless', 'baffled', 'staggered'],
    'friendship2': ['bond', 'companion', 'ally', 'trust', 'loyal', 'camaraderie', 'mate', 'bro', 'sisterhood', 'brotherhood',
                   'affection', 'partnership', 'support', 'understanding', 'team', 'respect', 'connect', 'harmony', 'together', 'kindred'],
    'jealousy2': ['envy', 'resent', 'covet', 'green-eyed', 'possessive', 'suspicion', 'mistrust', 'distrust', 'bitterness', 'spite',
                 'comparison', 'insecurity', 'longing', 'competitiveness', 'inferiority', 'grudge', 'fuming', 'anger', 'watchful', 'wary'],
    'nostalgia2': ['memories', 'past', 'bittersweet', 'longing', 'reminisce', 'yearning', 'flashback', 'old days', 'sentimental', 'melancholy',
                  'childhood', 'throwback', 'wistful', 'retro', 'antique', 'classic', 'golden days', 'vintage', 'heartwarming', 'timeless']
}

In [ ]:
topics = list(choices_dict.keys())# will be dict keys
counter = 0
next_topic=None


# iterate
for x in data['SurveyElements']:
    if type(x['Payload']) != dict:
        continue
    if 'QuestionType' not in x['Payload'].keys():
        continue
    if x['Payload']['QuestionType']=='MC': # group rank, influences others
        text = topics[counter]
        old_text = x['SecondaryAttribute']
        new_q = f'Select whether or not the following quotes are similar to the query "{text}"'
        x['Payload']['QuestionDescription']= new_q
        x['SecondaryAttribute'] = new_q
        x['Payload']['QuestionText']= new_q
        choices = choices_dict[text]
        new_choices = {str(x+1):{'Display': choices[x]} for x in range(len(choices))}
        x['Payload']['Choices']=new_choices
        x['Payload']['ChoiceOrder']=list(new_choices.keys())
        counter +=1
counter = 0
for x in data['SurveyElements']:
    if type(x['Payload']) != dict:
        continue
    if 'QuestionType' not in x['Payload'].keys():
        continue
    if x['Payload']['QuestionType']=='RO': # rank only, depends on other
        # THIS COMES FIRST
        # update Secondary Attribute: replace whatever is in ""
        text = topics[counter]
        next_topic = text
        old_text = x['SecondaryAttribute']
        new_q = f'Rank these quotes on how funny they are!'
        x['SecondaryAttribute'] = new_q
        # update ['Payload']['QuestionText'] replace whatever is in ""
        x['Payload']['QuestionText']= new_q
        x['Payload']['QuestionDescription']= new_q
        counter+=1
        #counter +=1

# save
with open('test.qsf', 'w') as json_file:
    json.dump(data, json_file) 

In [21]:
# make this a function

def new_survey_file(choices_dict:dict,file_name:str,template_name:str='Query_Template.qsf'):
    with open(template_name, 'r') as file:
        data = json.load(file)
    topics = list(choices_dict.keys())# will be dict keys
    c_topics = topics.copy()
    # surveyid = 'newstring'

    # iterate
    for x in data['SurveyElements']:
        if type(x['Payload']) != dict:
            continue
        if 'QuestionType' not in x['Payload'].keys():
            continue
        # x['SurveyID']= surveyid
        if x['Payload']['QuestionType']=='MC': # group rank, influences others
            text = topics.pop()
            new_q = f'Select whether or not the following quotes are similar to the query "{text}"'
            x['Payload']['QuestionDescription']= new_q
            x['SecondaryAttribute'] = new_q
            x['Payload']['QuestionText']= new_q
            choices = choices_dict[text]
            new_choices = {str(x+1):{'Display': choices[x]} for x in range(len(choices))}
            x['Payload']['Choices']=new_choices
            x['Payload']['ChoiceOrder']=list(new_choices.keys())
        if x['Payload']['QuestionType']=='RO': # rank only, depends on other
            # update Secondary Attribute: replace whatever is in ""
            text = c_topics.pop()
            new_q = f'Rank these quotes on how funny they are!'
            x['SecondaryAttribute'] = new_q
            # update ['Payload']['QuestionText'] replace whatever is in ""
            x['Payload']['QuestionText']= new_q
            x['Payload']['QuestionDescription']= new_q

    # save
    with open(file_name, 'w') as json_file:
        json.dump(data, json_file) 

### Load up examples

Now that I have the function written, I need to load up the search engine.

In [22]:
# load up that search
complete = pd.read_csv('https://scmcqueen.github.io/StarTrekScriptData/complete_data.csv')
complete.columns = ['index','character', 'quote', 'scene', 'location', 'view',
       'episode', 'date', 'series', 'file']
complete['quote']=complete['quote'].apply(lambda text: " ".join(text.split()))
complete['character']=complete['character'].fillna('NA')
test_engine = ss.search_engine()
test_engine.bulk_load(complete[['quote']].to_dict()['quote'])
test_engine.add_df(complete)

We have added 0 documents. The engine now has 144211 documents.
Complete data added.


In [6]:
results=test_engine.bw_search('candle',30)
search_results = test_engine.pretty_print([x[0] for x in results])

In [7]:
indices = [x[0] for x in results]
choices = [' '.join(x) for x in search_results]

In [8]:
keywords_1 =  ['candle','dream','doctor','bajor','prophets','inequality','struggle','engineer','prune','humanity']

In [ ]:
indices = []
choices = []
search_term = []
file_input = {}

for term in keywords_1:
    # do the search
    results=test_engine.bw_search(term,20)
    search_results = test_engine.pretty_print([x[0] for x in results])
    # update values
    t_ind = [x[0] for x in results]
    t_choices = [' '.join(x) for x in search_results]
    t_search = [term]*len(t_ind)
    # update big list
    indices+=t_ind
    choices+=t_choices
    search_term+=t_search
    # update the dict
    file_input[term] = t_choices  

new_survey_file(choices_dict=file_input,file_name='Evaluation/day_1.qsf',template_name='Query_Template.qsf')
pd.DataFrame({'indices':indices,'choices':choices,'search':search_term}).to_csv('Evaluation/day_1_data.csv')

In [32]:
keywords_1 =  ['candle','dream','doctor','bajor','prophets','inequality','struggle','engineer','prune','humanity',
               'story', 'religion', 'child', 'molly','counselor']
keywords_2 =  ['archaeology','chateau','trial','fiction','romance','wedding','father','kahless','chocolate','future',
               'bridge','saucer','coffee','romulans','colony']
keywords_3 =  ['raktajino','commit no errors','tanagra','borg','cardassia','poetry','spot','assimilate','unusual','barclay',
               'acquisition','rules','vedek','warp crystal',"bat'leth"]
keywords_4 =  ['blood wine','gagh','cook','celebrate','impossible','earl grey','jake','geordi','riker','poker',
               'phaser','occupation','conscious minds','tailor','neutral zone']
keywords_5 =  ['men','women','miles','worf','bashir','commander','captain','kira','troi','guinan',
               'shields','unification','mister spock','tribble','android']
keywords_6 =  ['unraveled','friendship','hailing frequencies','universe','traveler','wesley','good tea','good book','make it so','flute',
               'neutrino levels','noonian soong','breen','evidence','vacation']
keywords_7 =  ['memory','merry man','obedient','honor','san francisco','new orleans','liberation','imagination','maquis','tribunal',
               'trill','lore','risa','detective','ezri']
keywords_8 =  ['not picard','ice cream','morn','weyoun','dukat','civilization','replicators','truth','lunch','past',
               'empath','betray','warp bubble','jadzia','pregnant']
keywords_9 =  ['vineyard','lwaxana','bucket','justice is justice','espionage','same lie','mother','klingon','baseball','moriarty',
               'vulcan','moon','lower decks','the academy','full impulse']
keywords_10 =  ['enterprise','ferengi','nagus','death','root beer','alpha quadrant','wormhole','tasha','holosuite','holmes',
                'the defiant','ensign ro','dabo girl','secret agent','oo-mox']

In [ ]:
keywords = {1:keywords_1,
            2:keywords_2,
            3:keywords_3,
            4:keywords_4,
            5:keywords_5,
            6:keywords_6,
            7:keywords_7,
            8:keywords_8,
            9:keywords_9,
            10:keywords_10
            }

for x in keywords.keys():
    counter = str(x)
    t_keys = keywords[x]

    indices = []
    choices = []
    search_term = []
    file_input = {}

    for term in t_keys:
        # do the search
        results=test_engine.bw_search(term,20)
        search_results = test_engine.pretty_print([x[0] for x in results])
        # update values
        t_ind = [x[0] for x in results]
        t_choices = [' '.join(x) for x in search_results]
        t_search = [term]*len(t_ind)
        # update big list
        indices+=t_ind
        choices+=t_choices
        search_term+=t_search
        # update the dict
        file_input[term] = t_choices

    new_survey_file(choices_dict=file_input,file_name=f'Evaluation/day_{counter}.qsf',template_name='Query_Template.qsf')
    pd.DataFrame({'indices':indices,'choices':choices,'search':search_term}).to_csv(f'Evaluation/day_{counter}_data.csv')

# Upload Results


In [139]:
# start by reading a file
results = pd.read_csv('Evaluation/Results/day1_skyeler_only.tsv',sep='\t',encoding="utf-16")
questions_ids = pd.read_csv('Evaluation/day_1_data.csv')
results =results.drop(columns=['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage'])
cols = results.columns


In [ ]:
#           MY IDEAL OUTPUT
# NAME, QUERY, RANKING 1, RANKING 3, RANKING 3, RANKING 4, RANKING 5, .... 20

In [73]:
ranking_cols = [x for x in list(cols) if '_' in x]
results[ranking_cols]

,Q3_1,Q3_2,Q3_3,Q3_4,Q3_5,Q3_6,Q3_7,Q3_8,Q3_9,Q3_10,...,Q31_21,Q31_22,Q31_23,Q31_24,Q31_25,Q31_26,Q31_27,Q31_28,Q31_29,Q31_30
0,Rank these quotes on how funny they are! - RO:...,Rank these quotes on how funny they are! - QUA...,Rank these quotes on how funny they are! - TRO...,Rank these quotes on how funny they are! - BEV...,Rank these quotes on how funny they are! - LT....,Rank these quotes on how funny they are! - PIC...,Rank these quotes on how funny they are! - PUL...,Rank these quotes on how funny they are! - PUL...,Rank these quotes on how funny they are! - PUL...,Rank these quotes on how funny they are! - PUL...,...,Rank these quotes on how funny they are! - MUL...,Rank these quotes on how funny they are! - SIS...,Rank these quotes on how funny they are! - O'B...,Rank these quotes on how funny they are! - GUL...,Rank these quotes on how funny they are! - KIR...,Rank these quotes on how funny they are! - KAS...,Rank these quotes on how funny they are! - KIR...,Rank these quotes on how funny they are! - WIN...,Rank these quotes on how funny they are! - BAR...,Rank these quotes on how funny they are! - ODO...
1,"{""ImportId"":""QID4_1""}","{""ImportId"":""QID4_2""}","{""ImportId"":""QID4_3""}","{""ImportId"":""QID4_4""}","{""ImportId"":""QID4_5""}","{""ImportId"":""QID4_6""}","{""ImportId"":""QID4_7""}","{""ImportId"":""QID4_8""}","{""ImportId"":""QID4_9""}","{""ImportId"":""QID4_10""}",...,"{""ImportId"":""QID32_21""}","{""ImportId"":""QID32_22""}","{""ImportId"":""QID32_23""}","{""ImportId"":""QID32_24""}","{""ImportId"":""QID32_25""}","{""ImportId"":""QID32_26""}","{""ImportId"":""QID32_27""}","{""ImportId"":""QID32_28""}","{""ImportId"":""QID32_29""}","{""ImportId"":""QID32_30""}"
2,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,8,4,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN


In [161]:
q_col = [x for x in list(cols) if 'Q' in x and '_' not in x and 'Q1'!=x]
questions = results[q_col].T
questions = questions.reset_index()
questions['query']=questions[0].apply(lambda x:re.findall(r'"(.*?)"', x)[0])
query_map = questions[['index','query']]
query_map
# this can map the QID to the query

,index,query
0,Q2,prophets
1,Q4,doctor
2,Q6,dream
3,Q8,candle
4,Q10,counselor
5,Q12,molly
6,Q14,child
7,Q16,religion
8,Q18,story
9,Q20,humanity


In [ ]:
col_results = results[ranking_cols].T
col_results.dropna(inplace=True)
col_results.columns = ['question','id','ranking']
col_results = col_results.reset_index()
col_results['quote']=col_results['question'].apply(lambda x: x[x.index('-')+2:])
col_results['index'] = col_results['index'].apply(lambda x: x[:x.index('_')])

In [157]:
col_results['quote_id']=col_results['quote'].apply(lambda x: questions_ids[questions_ids.choices==x]['indices'].values[0])

In [193]:
map = {'Q3':'doctor',
       'Q5':'dream',
       'Q7':'candle',
       'Q9':'counselor',
       'Q11':'molly',
       'Q13':'child',
       'Q15':'religion',
       'Q17':'story',
       'Q19':'humanity',
       'Q21':'prune',
       'Q23':'engineer',
       'Q25':'struggle',
       'Q27':'inequality',
       'Q29':'prophets',
       'Q31':'bajor'
       }

In [ ]:
final_ranking=col_results.pivot_table(columns='ranking',index='index',values='quote_id',fill_value=0)
final_ranking=final_ranking.reset_index()
final_ranking['query']=final_ranking['index'].apply(lambda x: map[x])

final_ranking # for week 1

ranking,index,1,10,11,12,13,14,15,16,2,3,4,5,6,7,8,9,query
0,Q11,30991,0,0,0,0,0,0,0,31211,31072,31070,31085,30935,30998,0,0,molly
1,Q13,58776,0,0,0,0,0,0,0,132777,100032,100016,99902,93126,109712,137640,54925,child
2,Q15,32426,0,0,0,0,0,0,0,56079,126778,0,0,0,0,0,0,religion
3,Q17,12283,0,0,0,0,0,0,0,31013,140455,15569,68659,54951,0,0,0,story
4,Q19,117903,105556,138232,93373,140566,75783,79522,79073,98198,86230,61952,105445,138168,114115,105446,96561,humanity
5,Q21,90615,0,0,0,0,0,0,0,15974,13255,13254,90625,15975,13242,60004,64822,prune
6,Q23,31500,27739,34387,65602,0,0,0,0,34526,65552,56168,22503,30291,137508,65832,22638,engineer
7,Q25,96294,41720,0,0,0,0,0,0,107890,111497,52134,85549,48717,108567,46471,85548,struggle
8,Q29,43666,46548,42070,382,43823,1129,0,0,34729,38633,29191,43741,46598,13116,66498,43929,prophets
9,Q3,137655,124000,112964,0,0,0,0,0,121283,132030,138272,138539,138143,138077,138281,126612,doctor


In [245]:
def get_results_df(results_path:str='Evaluation/Results/day1_skyeler_only.tsv',questions_path:str='Evaluation/day_1_data.csv'):
    # start by reading a file
    results = pd.read_csv(results_path)
    questions_ids = pd.read_csv(questions_path)
    results =results.drop(columns=['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage'])
    cols = results.columns
    # get 
    ranking_cols = [x for x in list(cols) if '_' in x]
    q_col = [x for x in list(cols) if 'Q' in x and '_' not in x and 'Q1'!=x]
    questions = results[ranking_cols].T
    # query+map should be diff
    questions = questions.reset_index()
    questions['query']=questions[0].apply(lambda x:re.findall(r"'(.*?)'", x)[0])
    questions['index']=questions['index'].apply(lambda x: x[:x.index('_')])
    query_map = questions[['index','query']]
    # get rankings
    col_results = results[ranking_cols].T
    col_results.dropna(inplace=True)
    col_results.columns = ['question','id','ranking']
    col_results = col_results.reset_index()
    col_results['quote']=col_results['question'].apply(lambda x: x[x.index('-')+2:])
    col_results['index'] = col_results['index'].apply(lambda x: x[:x.index('_')])
    col_results['quote_id']=col_results['quote'].apply(lambda x: questions_ids[questions_ids.choices==x]['indices'].values[0])
    final_ranking=col_results.pivot_table(columns='ranking',index='index',values='quote_id',fill_value=0)
    final_ranking=final_ranking.reset_index()
    print(query_map.index)
          #[query_map.index=='Q3'])
    final_ranking['query']=final_ranking['index'].apply(lambda x: query_map[query_map['index']==x]['query'].values[0])
    return(final_ranking)

In [246]:
week2 =get_results_df(results_path='Evaluation/Results/day2_skyeler_only.csv',questions_path='Evaluation/day_2_data.csv')
week3 =get_results_df(results_path='Evaluation/Results/day3_skyeler_only.csv',questions_path='Evaluation/day_3_data.csv')
week4 =get_results_df(results_path='Evaluation/Results/day4_skyeler_only.csv',questions_path='Evaluation/day_4_data.csv')
week5 =get_results_df(results_path='Evaluation/Results/day5_skyeler_only.csv',questions_path='Evaluation/day_5_data.csv')

RangeIndex(start=0, stop=264, step=1)
RangeIndex(start=0, stop=296, step=1)
RangeIndex(start=0, stop=300, step=1)
RangeIndex(start=0, stop=273, step=1)


In [247]:
print(final_ranking.columns)
print(week2.columns)
print(week3.columns)
print(week4.columns)
print(week5.columns)

Index(['index', '1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4',
       '5', '6', '7', '8', '9', 'query'],
      dtype='object', name='ranking')
Index(['index', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'query'], dtype='object', name='ranking')
Index(['index', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5',
       '6', '7', '8', '9', 'query'],
      dtype='object', name='ranking')
Index(['index', '1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4',
       '5', '6', '7', '8', '9', 'query'],
      dtype='object', name='ranking')
Index(['index', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5',
       '6', '7', '8', '9', 'query'],
      dtype='object', name='ranking')


In [251]:
pd.melt(frame=week5,id_vars=['index','query'],value_vars=[ '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5',
       '6', '7', '8', '9'])

,index,query,ranking,value
0,Q11,tribble,1,12173
1,Q13,mister spock,1,12246
2,Q15,unification,1,94322
3,Q17,shields,1,102289
4,Q19,guinan,1,115444
...,...,...,...,...
190,Q3,miles,9,0
191,Q31,Worf,9,0
192,Q5,women,9,93075
193,Q7,men,9,0
